# Powerlifting Data Analysis

### Import Libraries

In [1]:
import pandas as pd

### Load Data Set
https://www.kaggle.com/datasets/dansbecker/powerlifting-database?select=meets.csv

In [2]:
df = pd.read_csv("data/openpowerlifting.csv")

### Get aquainted with data

In [7]:
df.columns

Index(['MeetID', 'Name', 'Sex', 'Equipment', 'Age', 'Division', 'BodyweightKg',
       'WeightClassKg', 'Squat4Kg', 'BestSquatKg', 'Bench4Kg', 'BestBenchKg',
       'Deadlift4Kg', 'BestDeadliftKg', 'TotalKg', 'Place', 'Wilks'],
      dtype='object')

In [8]:
df.shape

(386414, 17)

In [16]:
print(df.duplicated().sum() / df.shape[0])

df.loc[df.duplicated(keep=False)].sort_values(["BestBenchKg"])

df=df.drop_duplicates()

0.0014104043849342933


I'm noticing that the columns are pretty well standardized. There won't be too much cleaned needed, but the columns that end in 4kg seem to be empty, so I will do some more analysis and probably drop them.
About 0.14% of columns are duplicated, so we can drop all the duplicates without much more thought.

In [22]:
df["Bench4Kg"].notna().sum()
df["Squat4Kg"].notna().sum()
df["Deadlift4Kg"].notna().sum()

temp = df["Place"] == "1"
temp.sum()



194329

Less than 1% of columns have a value in the columns ending in 4Kg. They are going to be dropped.
I will also drop the "place" column because an unusually high percent (~50%) of columns say 1.

### Drop Columns

In [23]:
df = df.drop("Bench4kg", axis =1)
df = df.drop("Squat4Kg", axis =1)
df = df.drop("Deadlift4Kg", axis =1)
df = df.drop("Place", axis =1)

df.columns

KeyError: "['Bench4kg'] not found in axis"